## 내용정리
경상국립대학교 공지를 이용한 자연어처리 딥러닝 활용

각 공지가 어떠한 대상자에게 필요한지 분석 파악

대상자 : 대학생, 대학원생, 교수, 교직원, 일반인으로 라벨링

# KoBert 사용
## 필수 라이브러리 설치

In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

     |████████████████████████████████| 47.3 MB 1.5 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 4.2 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595736 sha256=70951b1ce984019a0e735da60e5d242d147564dbe0771d06418ecce49af0c9c6
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 4.3 MB/s 
     |████████████████████████████████| 769 kB 4.1 MB/s 
     |████████████████████████████████| 3.0 MB 58.3 MB/s 
     |████████████████████████████████| 895 kB 57.5 MB/s 


In [2]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-3pm4mp5y
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-3pm4mp5y
     |████████████████████████████████| 131 kB 4.1 MB/s 
     |████████████████████████████████| 4.5 MB 19.8 MB/s 
     |████████████████████████████████| 3.4 MB 57.3 MB/s 
     |████████████████████████████████| 3.3 MB 42.9 MB/s 
     |████████████████████████████████| 67 kB 5.0 MB/s 
     |████████████████████████████████| 596 kB 59.4 MB/s 
     |████████████████████████████████| 8.5 MB 51.2 MB/s 
     |████████████████████████████████| 79 kB 8.1 MB/s 
     |████████████████████████████████| 127 kB 80.4 MB/s 
  Created wheel for kobert: filename=kobert-0.2.3-py3-none-any.whl size=15449 sha256=8467457672420fbbe1c439382ad5419254dd0b019313440ee4bc4d3fb35e3b82
  Stored in directory: /tmp/pip-ephem-wheel-cache-62d4q12k/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd05

## Library patching

In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

import os


from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [5]:
device = torch.device("cpu")

In [6]:
bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


## Data preprocessing

In [14]:
import pandas as pd
aaa = pd.read_excel('/content/drive/MyDrive/bigdata_competition/sum_train.xlsx',index_col = 0)
bbb = pd.read_excel('/content/drive/MyDrive/bigdata_competition/validation_dt.xlsx',index_col = 0)

In [ ]:
aaa

,0,1,대학생,대학원생,교직원,교수,일반인
0,단체협약 체결을 위한 교섭요구 노동조합의 확정 공고,교섭요구 노동조합의 확정 공고 노동조합 및 노동관계조정법 시행령 규정에 따라 교섭요...,0,0,1,1,0
1,경상대 변경 통학 노선버스 운행시간,통학 노선버스 관련 운행시간 변동 사항 안내 자로 운행하는 통학 노선버스는 기존 운...,1,1,1,0,0
2,경상대 상반기 퇴직 공무원 정부포상 추천 후보자 공개검증,상반기 퇴직 공무원 포상 추천 후보자에 대해 아래와 같이 공개검증을 실시하오니 의견...,1,1,1,1,1
3,사회적 창업 아이디어 경진대회 참가자 모집 공고,지자체 대학 협력 기반 지역 혁신사업 경남지역 혁신 플랫폼 스마트 공동체 사업단 사...,1,1,0,0,0
4,재학생 등록금 납부 안내,재학생 등록금 추가 수납 일정을 아래와 같이 알려드리니 해당 학생들께서는 등록기간 ...,1,1,0,0,0
...,...,...,...,...,...,...,...
229,학생 인건비 통합기관 학생 인건비 지급 비율 및 부당 회수 비율 알림,학생 인건비 통합 관리기관인 경상 국립대학교 산학협력단의 학생 인건비 지급 비율 및...,1,1,1,1,1
230,경상 국립대학교 산학협력단 학생연구자 지원 규정 제정 알림,국가연구개발사업에 관한 범부처 공통규정으로 국가연구개발 혁신법이 제정되어 시행되었으...,1,1,1,1,1
231,기술 동향 조사 및 특허 맵 작성 수행기관 모집공고,기술 동향 조사 및 특허 맵 작성 수행기관 모집 공고 경상 국립대학교 산학협력단에서...,0,0,0,0,1
232,스마트 공장 사이니지 도입 확산 실증연구 실증 기업 모집,스마트 공장 사이니지 도입 확산 실증연구 실증 기업 모집 공고 중소 뿌리기업 제조 ...,0,0,0,0,1


In [15]:
train_set = aaa.iloc[:,1:]
valid_set = bbb.iloc[:,1:7]

In [ ]:
valid_set

,1,대학생,대학원생,교직원,교수,일반인
0,안녕하십니까 빅데이터 혁신 공유 대학 사업단입니다 경상 국립대학교가 참여 중인 빅데...,1,1,0,0,0
1,한국 교육 시설 안전원에서는 교육 시설 재난안전 관리의 중요성을 전파하고 학교 구성...,1,1,1,0,1
2,빅데이터 혁신 공유 대학 사업 빅데이터 명칭 공모전 빅데이터 혁신공유 대학 사업단에...,1,1,0,0,0
3,경남 마약퇴치운동 본부는 청년들의 마약퇴치 및 약물 오남용 에방에 대한 인식 제고와...,1,1,0,0,0
4,앙트러프러너십 교육 프로그램 국내 대학생 참가자 선발을 다음과 같이 안내하오니 관심...,1,0,0,0,0
...,...,...,...,...,...,...
7,재 재일한국인 본국투자 협회에서는 국내에서 수학하고 있는 재일 동포 학생들을 대상으...,1,0,0,0,0
8,선발 공고 농업인 자녀 장학생 장학생을 다음과 같이 선발하고자 하니 대학생 여러분의...,1,0,0,0,0
9,수산 훈계 장학금 지원내용 학기당 지원 대상 자격요건 신규자 해양수산계열 대학 해양...,1,0,0,0,0
10,대학생 청소년교육지원 사업 멘토 대체 추가 선발 결과 안내 대학생 청소년교육지원 사...,1,0,0,0,0


In [16]:
lst2 = []
for i in range(train_set.shape[0]):
    lst = []
    lst.append(train_set.iloc[i,0])
    lst.append(train_set.iloc[i,1:].tolist())
    lst2.append(lst)

In [17]:
lst4 = []
for i in range(valid_set.shape[0]):
    lst3 = []
    lst3.append(valid_set.iloc[i,0])
    lst3.append(valid_set.iloc[i,1:].tolist())
    lst4.append(lst3)

In [ ]:
len(lst2), len(lst2[0]), len(lst2[0][1])

(1134, 2, 5)

In [ ]:
len(lst4), len(lst4[0]), len(lst4[0][1])

(92, 2, 5)

In [ ]:
# ### 데이터 불러오기

# dataset_train = []
# dataset_test = []

# root = "/content/drive/MyDrive/newsData/"
# list = os.listdir(root)
# for cat in list:
#     files = os.listdir(root + cat)
#     for i,f in enumerate(files):
#         fname = root + cat + "/" + f
#         file = open(fname, "r")
#         strings = file.read()
#         if i<170:
#             dataset_train.append([strings, cat])
#         else:
#             dataset_test.append([strings,cat])
#         file.close()

# print(len(dataset_train), len(dataset_test))

1360 240


## Make KoBert

In [18]:
### 토큰나이저 사용
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [7]:
### 딥러닝용 데이터 셋으로 전환
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.float32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [8]:
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [19]:
data_train = BERTDataset(lst2, 0, 1, tok, max_len, True, False)
data_valid = BERTDataset(lst4, 0, 1, tok, max_len, True, False) ## data_test

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=2, shuffle=True)
valid_dataloader = torch.utils.data.DataLoader(data_valid, batch_size=batch_size, num_workers=5, shuffle=True)   ## test_dataloader

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
data_train.labels

In [ ]:
data_valid.labels

In [ ]:
next(iter(train_dataloader))

In [9]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=5,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device), return_dict=False)

        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out) 

## predict Multi-Label, Change loss-fucntion and calcuate accuarate function

In [20]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.BCEWithLogitsLoss()


t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
# def calc_accuracy(X,Y):
#     max_vals, max_indices = torch.max(X, 1)
#     train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
#     return train_acc

In [10]:
def calc_accuracy(X,Y):
    prob = torch.sigmoid(X)
    pred = (prob>0.5).type(torch.FloatTensor)
    correct = (pred == Y).type(torch.FloatTensor)
    return float(torch.mean(correct))

## Training

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    valid_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        label = label.float()
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()
        train_acc += calc_accuracy(out, label.float())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(valid_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        valid_acc += calc_accuracy(out, label.float())
    print("epoch {} valid acc {}".format(e+1, valid_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/18 [00:00<?, ?it/s]

epoch 1 train acc 0.6554649737146165


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/2 [00:00<?, ?it/s]

...

epoch 4 valid acc 0.9024553596973419


  0%|          | 0/18 [00:00<?, ?it/s]

epoch 5 train acc 0.9101600282722049


  0%|          | 0/2 [00:00<?, ?it/s]

epoch 5 valid acc 0.8948660790920258


## Save Training result

In [ ]:
torch.save(model.state_dict(),'/content/drive/MyDrive/bigdata_competition/model_state_dict.pt')

In [ ]:
torch.save(model,'/content/drive/MyDrive/bigdata_competition/model.pt' )

In [ ]:
torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict()
}, '/content/drive/MyDrive/bigdata_competition/model.ptall.tar')

## Predict Test set

In [25]:
def softmax(vals, idx):
    valscpu = vals.cpu().detach().squeeze(0)
    a = 0
    for i in valscpu:
        a += np.exp(i)
    return ((np.exp(valscpu[idx]))/a).item() * 100

def testModel(model, seq):
    cate = ["대학생","대학원생","교수", "교직원","일반인"]
    tmp = [seq]
    transform = nlp.data.BERTSentenceTransform(tok, max_len, pad=True, pair=False)
    tokenized = transform(tmp)

    # modelload.eval()
    result = model(torch.tensor([tokenized[0]]).to(device), [tokenized[1]], torch.tensor(tokenized[2]).to(device))
    idx = result.argmax().cpu().item()
    print("뉴스의 카테고리는:", cate[idx])
    print("신뢰도는:", "{:.2f}%".format(softmax(result,idx)))
    return result

In [ ]:
cate =["대학생","대학원생","교수", "교직원","일반인"]
cate[(torch.sigmoid(ppp)>0.5).type(torch.FloatTensor).tolist()]

TypeError: ignored

In [ ]:
torch.sigmoid(ppp)

tensor([[0.8791, 0.6277, 0.1475, 0.1297, 0.3195]], grad_fn=<SigmoidBackward0>)

In [ ]:
(torch.sigmoid(ppp)>0.5).type(torch.FloatTensor).tolist()[0]

[1.0, 1.0, 0.0, 0.0, 0.0]

In [ ]:
ppp = testModel(model, "지자체 대학 협력 기반 지역 혁신사업 경남지역 혁신 플랫폼 스마트 공동체 사업단 사회적 창업 아이디어 경진대회 참가자 모집 공고 경남지역 혁신 플랫폼 스마트 공동체 사업단에서는 창의적인 아이디어로 사회적 가치를 창출하는 소셜벤처 아이디어를 발굴하고 사업화를 지원하는 사회적 창업 아이디어 경진대회의 참 가자를 다음과 같이 모집하오니 많은 참가 바랍니다 지원 대상 경상대학교 경남대학교 창원대학교 인제대학교 학부 및 대학원 재학생 창업 동아리 예비 창업자 창업 미만 창업자 및 창업에 열정과 관심 있는 모든 학생 개인 혹은 팀으로 신청 팀 구성원은 미만 선발 규모 및 상금 총상금 최우수상 우수상 신청 기간 공고일 추진일정 및 기타 사항 첨부자료 참조 사회적 창업 아이디어 경진대회 신청방법 이메일 접수 문의처 담당자 경남지역 혁신 플랫폼 스마트 공동체 사업단 창업 소 과제 담당 조윤정 주무관 연 락 처 진행 상황에 따라 추후 대회 및 심사 일정은 변경될 수 있음")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  from ipykernel import kernelapp as app


뉴스의 카테고리는: 대학생
신뢰도는: 74.59%


In [ ]:
ppp

tensor([[ 1.9842,  0.5225, -1.7541, -1.9039, -0.7562]],
       grad_fn=<AddmmBackward0>)

In [ ]:
pong=np.array([1,2,3,4,5])
pong[True,False,True,False,True]

array([], shape=(0, 5), dtype=int64)

In [21]:
PATH = '/content/drive/MyDrive/bigdata_competition/'
model = torch.load(PATH + 'model.pt')  # 전체 모델을 통째로 불러옴, 클래스 선언 필수
model.load_state_dict(torch.load(PATH + 'model_state_dict.pt'))  # state_dict를 불러 온 후, 모델에 저장

checkpoint = torch.load(PATH + 'model.ptall.tar')   # dict 불러오기
model.load_state_dict(checkpoint['model'])
optimizer.load_state_dict(checkpoint['optimizer'])

In [23]:
ddd = pd.read_excel('/content/drive/MyDrive/bigdata_competition/test_data_clean_version2.xlsx',index_col=0)
ddd.head()

,0,1,대학생,대학원생,교직원,교수,일반인
0,지속 가능발전 목표 인식 제고를 위한 교육 개최 교직원 대상,우리 대학 교직원의 시민의식 향상과 인식 제고를 위한 교육을 다음과 같이 개최하오니...,0,0,1,0,0
1,하반기 일본 파견 교환학생 선발 공고,하반기에 일본 대학으로 파견할 교환학생 후보자를 다음과 같이 선발하오니 관심 있는 ...,1,0,0,0,0
2,국립대학 육성사업 재학생 역량 강화를 위한 겨울방학 특강,정보 전 산처에서는 국립대학 육성사업으로 재학생의 역량 강화를 위한 겨울방학 특강을...,1,0,0,0,0
3,학생생활관 가좌캠퍼스 생활지도원 모집 공고,학생생활관 가좌캠퍼스의 관생 지도를 담당할 생활지도원을 모집하니 관심 있는 대학원생...,0,1,0,0,0
4,홍보 풍류 대장 전국투어 콘서트 진주 교직원 및 학생 특별 할인 안내,안녕하세요 단계적 일상 회복 전환에 따른 사회적 거리두기 지침이 개편됨에 따라 전공...,1,1,1,1,0


In [26]:
kilo = []
for i in ddd.iloc[:,1]:
    ppp = testModel(model, i)
    kilo.append((torch.sigmoid(ppp)>0.5).type(torch.FloatTensor).tolist()[0])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  from ipykernel import kernelapp as app


뉴스의 카테고리는: 대학생
신뢰도는: 37.52%
뉴스의 카테고리는: 대학생
신뢰도는: 87.46%
뉴스의 카테고리는: 대학생
신뢰도는: 87.75%
뉴스의 카테고리는: 대학생
신뢰도는: 87.73%
뉴스의 카테고리는: 대학생
신뢰도는: 50.10%
뉴스의 카테고리는: 대학생
신뢰도는: 77.90%
뉴스의 카테고리는: 대학생
신뢰도는: 85.30%
뉴스의 카테고리는: 대학생
신뢰도는: 86.26%
뉴스의 카테고리는: 대학생
신뢰도는: 85.22%
뉴스의 카테고리는: 일반인
신뢰도는: 52.29%
뉴스의 카테고리는: 대학생
신뢰도는: 79.91%
뉴스의 카테고리는: 대학생
신뢰도는: 85.14%
뉴스의 카테고리는: 대학생
신뢰도는: 83.99%
뉴스의 카테고리는: 일반인
신뢰도는: 93.22%
뉴스의 카테고리는: 일반인
신뢰도는: 92.62%
뉴스의 카테고리는: 일반인
신뢰도는: 93.24%
뉴스의 카테고리는: 일반인
신뢰도는: 93.30%
뉴스의 카테고리는: 일반인
신뢰도는: 93.08%
뉴스의 카테고리는: 일반인
신뢰도는: 93.25%
뉴스의 카테고리는: 일반인
신뢰도는: 93.30%
뉴스의 카테고리는: 일반인
신뢰도는: 93.01%


In [27]:
pd.DataFrame(kilo).to_excel('test_result_version2.xlsx')